# CTOからの挑戦状 2014

- $m$ クーポンの種類 $i = 1,....m$
- $CouponPrice_i$ クーポンの価格
- $CouponPosessions_i$ クーポンの手持ち数
- $CouponUse_i \geq 0$ クーポンの利用枚数
- $TotalPrice$ 支払い金額

制約
- $\sum_i CouponPrice_iCouponUse_i \leq TotalPrice$
- $CouponUse_i \leq CouponPosessions_i$

In [1]:
import cvxpy

In [49]:
m = 3
coupon_price = [50, 100, 500]

In [52]:
def solve(coupon_posessions, total_price):
    """
    金額と手持ちのクーポンから、どのクーポンを何枚使うかを返す
    
    Params
    ------
    coupon_posessions: list[int]
        クーポン所持数
    total_price: int
        購入金額
        
    Returns
    -------
    coupon_use: list[int]
        クーポン毎の使用数
    """
    coupon_use = cvxpy.Variable(m)
    coupon_amount = coupon_use.T*coupon_price
    
    # 少ないクーポンでクーポン使用合計金額を最大化する
    subject = coupon_amount - sum(coupon_use)
    prob = cvxpy.Problem(cvxpy.Maximize(subject), [
               coupon_amount <= total_price,    # 合計額を越えてクーポンは使えない
               coupon_use == cvxpy.Int(m),      # クーポン使用数は整数
               coupon_use >= 0,                 # クーポン使用数はゼロ以上
               coupon_use <= coupon_posessions, # クーポン使用数は手持ちクーポン以下
               sum(coupon_use) == cvxpy.Int(1)
              ])
    prob.solve()
    # 0.999999とか出るので近い整数に丸める
    return np.rint(coupon_use.value.A.flatten()).astype(int).tolist()

## テストケース

In [51]:
assert(solve([0,0,0], 100) == [0,0,0])
assert(solve([2,1,0], 100) == [0,1,0])
assert(solve([3,5,1], 470) == [1,4,0])
assert(solve([6,5,2], 1230) == [0,2,2])